In [1]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import datetime as dt
from sklearn.metrics import f1_score
from sklearn import linear_model

In [7]:
smpl_sub = pd.read_csv('sample_submission.csv',index_col='ID')
item = pd.read_csv('items.csv')
s_train = pd.read_csv('sales_train.csv')
s_train['item_cnt_day'] = s_train['item_cnt_day'].astype('int')
# s_train['date'] = pd.to_datetime(s_train['date']).map(dt.datetime.toordinal)
# s_train = s_train.drop(['date','date_block_num'],axis = 1)

In [5]:
# dt.datetime.toordinal(pd.to_datetime(s_train['date']))
# s_train.loc[2,'date']
# s_train[(s_train['item_cnt_day']==2.0)]

In [4]:
test = pd.read_csv('test.csv')
test = test.set_index('ID')
it_cat = pd.read_csv('item_categories.csv')
shop = pd.read_csv('shops.csv')
s_train = s_train.drop(['date'],axis = 1)
s_train.head(3)

In [27]:
# import sys
# !{sys.executable} -m pip install pandas-profiling

import pandas_profiling as pp
le = preprocessing.LabelEncoder()
s_train
# item.head(3)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1
1,03.01.2013,0,25,2552,899.00,1
2,05.01.2013,0,25,2552,899.00,-1
3,06.01.2013,0,25,2554,1709.05,1
4,15.01.2013,0,25,2555,1099.00,1
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1
2935845,09.10.2015,33,25,7460,299.00,1
2935846,14.10.2015,33,25,7459,349.00,1
2935847,22.10.2015,33,25,7440,299.00,1


In [26]:
# gg = it_cat.copy()
it_cat.head(3)
# le.fit(it_cat['item_category_name'])
# gg['no'] = le.transform(it_cat['item_category_name'])
# gg
# pd.get_dummies()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2


In [7]:
cd = test.copy()
cd['item_cnt_month'] = -1000.9
cd['']
test.head(3)

,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233


In [8]:
# test[(test['item_id']==5320)]
# len(cd)
# ef = s_train[['shop_id','item_id','date_block_num','item_price']]
# ef_grp = ef.groupby(['shop_id','item_id'])
# cd

In [9]:
shop.head(3)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2


In [10]:
smpl_sub.head(3)

,item_cnt_month
ID,
0,0.5
1,0.5
2,0.5


In [11]:
# s_train[(s_train['item_cnt_day']>5)]

In [12]:
# sgroup = s_train.groupby(['shop_id','item_id'])
# tgroup = test.groupby('shop_id')

In [13]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelBinarizer
# type(sgroup.get_group(5))
# tgroup.get_group(1)
# y = LabelBinarizer(ab.item_name)
# print(y)
# ab.corr()
sgroup = s_train.groupby('item_id')
tgroup = test.groupby('shop_id')

In [17]:
# sgroup.get_group((5,5320))
n = sgroup.groups.keys()
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=4, max_depth=3)

In [15]:
# for i in range(len(cd)):
#     a = test.loc[i,'shop_id']
#     b = test.loc[i,'item_id']
#     group = sgroup.get_group((a,b))
#     if (a in test.shop_id.unique())&(b in tgroup.get_group(a).item_id):
#         if len(group)>2:
#             neigh = KNeighborsClassifier(n_neighbors = 2)
#             X = preprocessing.StandardScaler().fit_transform(group[['date_block_num','item_price']].values)
#             X_train, X_test, y_train, y_test = train_test_split( X, group['item_cnt_day'].values, test_size=0, shuffle = False)
#             neigh.fit(X_train,y_train)
#             v = neigh.predict([[ef_grp.get_group((a,b))['date_block_num'].max()+1,ef_grp.get_group((a,b))['item_price'].mean()]])
#             cd.loc[i,'item_cnt_month'] = v
# #           print(f1_score(y_test, neigh.predict(X_test)))
# #         elif 
# #         else:
# #           print(f1_score(y_test, group2['item_cnt_day']))

In [20]:
for i in range(len(test)):
    a = test.loc[i,'shop_id']
    b = test.loc[i,'item_id']
    if b in n:
        group = sgroup.get_group(b)
        if len(group)>=4:
#             neigh = KNeighborsClassifier(n_neighbors = 4,weights = 'distance')
            regr = linear_model.LinearRegression()
            X_train = group[['shop_id','date_block_num','item_price']].values
            y_train = group['item_cnt_day'].values
#             neigh.fit(X_train,y_train)
#             regr.fit(X_train,y_train)
            model.fit(X_train,y_train)
            if a in group['shop_id'].unique():
                if len(group[group['shop_id']==a])>2:
                    regr.fit(group[group['shop_id']==a][['date_block_num']].values,group[group['shop_id']==a]['item_price'].values)
                    v = np.round(model.predict([[a, 34, regr.predict([[34]])]]))
                else:
                    v = np.round(model.predict([[a, 34, group[group['shop_id']==a]['item_price'].mean()]]))
                if v<-1:
                    v = -1
                cd.loc[i,'item_cnt_month'] = v
                print(v," first ")
            else:
#                 v = neigh.predict([[a,group['item_price'].mean()]])
                cd.loc[i,'item_cnt_month'] = 0
                print(0," second ")
        else:
            cd.loc[i,'item_cnt_month'] = round(group['item_cnt_day'].mean())
            print(round(group['item_cnt_day'].mean())," third ")
#             print(group['item_cnt_day'].mean())
    else:
#         v = s_train[s_train['shop_id']==a]['item_cnt_day'].mean()
        cd.loc[i,'item_cnt_month'] = 0
        print(0," fourth ")

[1]  first 
0  fourth 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
0  fourth 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  fi

[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
0  second 
0  fourth 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[2]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  fourth 
[2]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  fourth 
0  fourth 
0  fourth 
[1]  first 
0 

[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
1  third 
[1]  first 
0  fourth 
[1]  first 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
0  fourth 
0  second 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
1  third 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[-1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1] 

[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
0  second 
[1]  first 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
[1]  first 
0  second 
1  third 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
0  fourth 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
0  second 
0  second 
0  second 
0  second 
0  second 
0  fourth 
0  second 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
0  second 
[

0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
0  second 
0  second 
0  second 
0  second 
0  second 
[1]  first 
0  second 
0  second 
[1]  first 
0  second 
0  second 
0  second 
0  second 
0  fourth 
[1]  first 
0  second 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[2]  first 
1  third 
0  second 
0  second 
0  second 
0  fourth 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 

...

[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
1  third 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
1  third 
0  second 
[1]  first 
[1]  first 
[1]  first 
2  third 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
0  fourth 
0  fourth 
0  fourth 
[1]  first 
0  second 
0  second 
0  fourth 
[1]  first 
[1]  

0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  fourth 
0  second 
[1]  first 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
1  third 
[1]  first 
0  second 
0  second 
0  second 
0  second 
[1]  first 
0  second 
0  second 
0  second 
0  second 
[1]  first 
0  second 
0  second 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  fo

[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
1  third 
0  second 
0  second 
[1]  first 
0  fourth 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
1  third 
[1]  first 
0  second 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
0  fourth 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
[1]  firs

0  second 
0  second 
0  second 
0  second 
0  second 
0  fourth 
0  second 
0  second 
1  third 
0  second 
0  second 
0  second 
0  fourth 
0  second 
1  third 
0  second 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
0  second 
1  third 
0  second 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
0  fourth 
[1]  first 
[1]  first 
[1]  first 
0  second 
0  second 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[1]  first 
[3]  first 
0  fourth 
0  second 
[1]  first 
[1]  first 
[1]  first 
0  second 
[1]  first 
0  second 
[1]  first 
0  second 
0  second 
[1]  first 


0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
1  third 
0  fourth 
0  fourth 
1  third 
0  second 
0  second 
1  third 
0  second 
0  second 
0  second 
0  second 
0  second 
2  third 
0  second 
0  second 
0  second 
0  second 
0  second 
0  fourth 
0  second 
0  second 
0  second 
0  fourth 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  fourth 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
1  third 
1  third 
0  second 
0  second 
0  second 
1  third 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
0  second 
[1]  first 
0  second 
0  second 
0  second 
0  se

In [21]:
de = cd[['item_cnt_month']]

In [22]:
de['item_cnt_month'] = round(de['item_cnt_month'])
de

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,item_cnt_month
ID,
0,1.0
1,0.0
2,1.0
3,1.0
4,0.0
5,1.0
6,1.0
7,1.0
8,1.0


In [23]:
de.to_csv('submssn6.csv')

In [28]:
# de[de['item_cnt_month']==0]

In [ ]:
_